1. Oxonium MS/MS filtrering 

In [ ]:
import pandas as pd
from pyteomics import mgf, pylab_aux
import pylab
import math

def filter_mgf(source, seed_mass=204.09, tol=0.01):
    filtered_spectra=[]
    spectra = mgf.read(source)
    for i in range(len(spectra)):
        spectrum = next(spectra)
        if any([math.isclose(seed_mass, k, abs_tol=tol) for k in spectrum['m/z array']]):
            filtered_spectra.append(spectrum)
    return filtered_spectra

In [ ]:
files = [] # list of MGF files
for file in files:
    mgf.write(filter_mgf(source=file), output=file.replace('.mgf', '_filtered_001.mgf'))

2. Byonics search + METABOLOMICS-SNETS-V2 output Network walk

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import math
import json

In [ ]:
node_data = pd.read_csv('', index_col='shared name',low_memory=False) # METABOLOMICS-SNETS-V2 output node information
node_data = node_data.loc[node_data['number of spectra']>=3]
edge_data = pd.read_csv('',low_memory=False) # METABOLOMICS-SNETS-V2 output output edge information

data = pd.read_excel('') # Byonics search data
data = data.loc[~data['Glycan Composition'].str.contains('Na')]

In [ ]:
G = nx.DiGraph()
for index in edge_data.index:
    if edge_data.loc[index]['node1'] != edge_data.loc[index]['node2'] \
    and edge_data.loc[index]['node1'] in node_data.index \
    and edge_data.loc[index]['node2'] in node_data.index:
        node1_mass = node_data.loc[edge_data.loc[index]['node1']]['parent mass']
        node2_mass = node_data.loc[edge_data.loc[index]['node2']]['parent mass']
        mass_difference = edge_data.loc[index]['mass_difference']
        if mass_difference < 0:
            if node1_mass >= node2_mass:
                node1=edge_data.loc[index]['node1']
                node2=edge_data.loc[index]['node2']
            else:
                node1=edge_data.loc[index]['node2']
                node2=edge_data.loc[index]['node1']
        if mass_difference >= 0:
            if node1_mass >= node2_mass:
                node1=edge_data.loc[index]['node2']
                node2=edge_data.loc[index]['node1']
            else:
                node1=edge_data.loc[index]['node1']
                node2=edge_data.loc[index]['node2']
        G.add_edge(node1, node2, mass_difference=mass_difference)
nx.set_node_attributes(G, {i[0]:{'parent mass':i[1], 'precursor mass':i[2], 'number of spectra':i[3], 'UniqueFileSources':i[4]} for i in zip(node_data.index,
                                                                                                                                             node_data['parent mass'],
                                                                                                                                             node_data['precursor mass'],
                                                                                                                                             node_data['number of spectra'],
                                                                                                                                             node_data['UniqueFileSources'])})

node_parent_mass_d = {i[1]['parent mass']:i[0]for i in list(G.nodes(data=True))}
count=0
for seed_mass, seed_pep, seed_glycan in zip(data['Theo. MH+ [Da]'], data['Annotated Sequence'], data['Glycan Composition']):
    for k,v in node_parent_mass_d.items():
        if math.isclose(seed_mass, k, abs_tol=1.10):# corrected
            count+=1
            try:
                G.nodes[v]['glycoform'].append((seed_pep, seed_glycan))
                #print( G.nodes[v], v, seed_pep, seed_glycan, node_parent_mass_d[k])
            except:
                G.nodes[v].update({'glycoform':[(seed_pep, seed_glycan)]})

print(nx.info(G))

In [ ]:
def glycan_motifs(glycan):
    motifs = {}
    for a in glycan.split(')')[:-1]:
        motifs[a.split('(')[0]]=int(a.split('(')[1])
    return motifs

from itertools import combinations_with_replacement

def find_possible_glycoforms(seed_glycoform, mass_diff, n, mono_mass, tol=0.10):
    new_glycans=[]
    mono_mass_rev = {v:k for k,v in mono_mass.items()}
    num = list(mono_mass.values()) + [-1*i for i in mono_mass.values()]
    for m in range(1,n+1):
        for masses in combinations_with_replacement(num,m):
            if math.isclose(mass_diff, sum(masses), abs_tol=tol) and [abs(i) for i in masses].count(1.0)<=1:
                foo=True
                test = list(zip(masses, [mono_mass_rev[abs(j)] for j in masses]))
                for index in range(len(test)):
                    if foo:
                        for j_index in range(len(test[index+1:])):
                            if sum((test[index][0], test[index+1+j_index][0]))==0:
                                foo=False
                                break
                    else:
                        break
                if not foo:
                    pass
                else:
                    bar = True
                    motifs = glycan_motifs(seed_glycoform)
                    motifs.update({'HYDRO':1.0})
                    for item, val in zip([mono_mass_rev[abs(j)] for j in masses], masses):
                        if val > 0:
                            try:
                                motifs[item]+=1
                            except:
                                motifs.update({item:1})
                        elif val < 0:
                            try:
                                motifs[item]-=1
                            except:
                                bar=False
                                break
                    if bar:
                        motifs.pop('HYDRO', None)
                        order = {'HexNAc':0, 'Hex': 1, 'Fuc': 2, 'NeuAc': 3, 'NeuGc': 4}
                        new_glycan = ''.join(['{}({})'.format(i,j) for i,j in sorted(motifs.items(), key=lambda x: order[x[0]]) if j!=0])
                    elif not bar:
                        new_glycan = ''
                    if '-' in new_glycan or len(new_glycan)==0:
                        break
                    new_glycans.append(new_glycan)
    if mass_diff <= tol and mass_diff >= 0:
        new_glycans.append(seed_glycoform)
    return new_glycans

In [ ]:
# go through nodes until len new_nodes == 0 new nodes added if brand new or mutated by addition

n=2
#mono_mass = {'HexNAc':203.080,'Hex':162.053,'Fuc':146.058,'NeuAc':291.095,'HYDRO':1.0}
mono_mass = {'HexNAc':203.080,'Hex':162.053,'Fuc':146.058,'NeuAc':291.095,'NeuGc':307.090,'HYDRO':1.0}

orig_nodes = [k for k,v in G.nodes(data=True) if 'glycoform' in v.keys()]
new_nodes = orig_nodes
edges_done = []
while len(new_nodes) != 0:
    prev_nodes = new_nodes
    new_nodes = list()
    for node in prev_nodes:
        for edge in nx.edges(G,node):
            new_node = edge[1]
            if new_node not in orig_nodes:
                mass_diff = G.edges[edge]['mass_difference']
                if (mass_diff < 0 and G.nodes(data=True)[node]['parent mass'] > G.nodes(data=True)[new_node]['parent mass']) or \
                (mass_diff >= 0 and G.nodes(data=True)[node]['parent mass'] <= G.nodes(data=True)[new_node]['parent mass']):
                    if 'glycoform' in G.nodes[node].keys():
                        for seed_pep, seed_glycan in G.nodes[node]['glycoform']:
                            new_glycans = find_possible_glycoforms(seed_glycoform=seed_glycan, mass_diff=mass_diff, n=n, mono_mass=mono_mass)
                            if len(new_glycans) != 0:
                                #print(node, new_node, seed_pep, seed_glycan, new_glycans, mass_diff)
                                #print()
                                for new_glycan in new_glycans:
                                    try:
                                        if (seed_pep, new_glycan) not in G.nodes[new_node]['glycoform']:
                                            G.nodes[new_node]['glycoform'].append((seed_pep, new_glycan))
                                            new_nodes.append(new_node)
                                    except:
                                        G.nodes[new_node].update({'glycoform':[(seed_pep, new_glycan)]})
                                        new_nodes.append(new_node)
        # all nodes coming in
        for inbound_node in [i[0] for i in G.edges() if i[1]==node]:
            new_node = inbound_node
            if 'glycoform' in G.nodes[new_node].keys():
                if len(set([i[0] for i in G.nodes[new_node]['glycoform']]).intersection(set([i[0] for i in G.nodes[node]['glycoform']]))) > 0:
                    continue    
            for edge in [i for i in nx.edges(G,new_node) if i[1]==node]:
                if new_node not in orig_nodes:
                    mass_diff = G.edges[edge]['mass_difference']
                    if (mass_diff < 0 and G.nodes(data=True)[new_node]['parent mass'] > G.nodes(data=True)[node]['parent mass']) or \
                    (mass_diff >= 0 and G.nodes(data=True)[new_node]['parent mass'] <= G.nodes(data=True)[node]['parent mass']):
                        if 'glycoform' in G.nodes[node].keys():
                            for seed_pep, seed_glycan in G.nodes[node]['glycoform']:
                                new_glycans = find_possible_glycoforms(seed_glycoform=seed_glycan, mass_diff=-1*(mass_diff), n=n, mono_mass=mono_mass)
                                if len(new_glycans) != 0:
                                    #print(node, new_node, seed_pep, seed_glycan, new_glycans, mass_diff)
                                    #print()
                                    for new_glycan in new_glycans:
                                        try:
                                            if (seed_pep, new_glycan) not in G.nodes[new_node]['glycoform']:
                                                G.nodes[new_node]['glycoform'].append((seed_pep, new_glycan))
                                                new_nodes.append(new_node)
                                        except:
                                            G.nodes[new_node].update({'glycoform':[(seed_pep, new_glycan)]})
                                            new_nodes.append(new_node)

In [ ]:
nx.to_pandas_edgelist(G).to_csv('', index=False)

nodelist_data = pd.DataFrame.from_dict(G.nodes, orient='index')
seed_d = {}
for index in nodelist_data.loc[nodelist_data.notna()['glycoform']].index:
    if index in orig_nodes:
        seed_d[index]='Y'
    else:
        seed_d[index]='N'
nodelist_data['Byonics Seed'] = pd.Series(seed_d)

accession_map = dict(zip(data['Annotated Sequence'], data['Master Protein Accessions']))
accession_d = {}
for index in nodelist_data.loc[nodelist_data.notna()['glycoform']].index:
    accession_d[index] = [accession_map[i[0]] for i in nodelist_data.loc[index]['glycoform']]
nodelist_data['Protein Accession'] = pd.Series(accession_d)
nodelist_data.to_csv('')

3. Glycan Fragmentation Network

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('', index_col=0) # nodelist_data from previous section
data = data.dropna(subset=['glycoform'])

glycans = set([i[1] for j in data['glycoform'] for i in eval(j)])
glycans = (glycans - set([i for i in glycans if 'HexNAc' not in i]))

In [ ]:
def glycan_motifs(glycan):
    motifs = {}
    for a in glycan.split(')')[:-1]:
        motifs[a.split('(')[0]]=int(a.split('(')[1])
    return motifs

def tree_builder(mod_motifs):
    tree = {}
    for k,v in mod_motifs.items():
        for k2,v2 in mod_motifs.items():
            bar=False
            if k!=k2:
                if len(v2)==len(v):
                    foo=[]
                    for mod in v:
                        if mod[0] in [i[0] for i in v2]:
                            foo.append(True)
                        else:
                            foo.append(False)
                    if sum(foo)==len(v2):
                        foo=[]
                        for mod in v:
                            if mod[1] == [i[1] for i in v2 if mod[0]==i[0]][0]:
                                foo.append(True)
                            elif mod[1] == str(int([i[1] for i in v2 if mod[0]==i[0]][0])-1):
                                foo.append(False)
                            else:
                                for x in range(len(v2)*2):
                                    foo.append(True)
                        if sum(foo)==len(v2)-1:
                            if k2 in tree.keys():
                                tree[k2].append(k)
                                bar=True
                            else:
                                tree[k2]=[]
                                tree[k2].append(k)
                                bar=True
                if len(v2)-len(v)==1:
                    foo=[]
                    for mod in v:
                        if mod[0] in [i[0] for i in v2]:
                            foo.append(True)
                        else:
                            foo.append(False)
                    if [i for i in v2 if i[0] not in [i[0] for i in v]][0][1]==str(1):
                        if sum(foo)==len(v):
                            foo=[]
                            for mod in v:
                                if mod[1] == [i[1] for i in v2 if mod[0]==i[0]][0]:
                                    foo.append(True)
                                else:
                                    foo.append(False)
                        if sum(foo)==len(v):  
                            if k2 in tree.keys():
                                tree[k2].append(k)
                                bar=True
                            else:
                                tree[k2]=[]
                                tree[k2].append(k)
                                bar=True
    return tree

In [ ]:
mod_motifs = {}
for glycan in glycans:
    mod_motifs[glycan]=[[k, str(v)] for k,v in glycan_motifs(glycan).items()]
tree = tree_builder(mod_motifs)

In [ ]:
while len(glycans - set(tree.keys())) > 1:
    new_glycans = set()
    for glycan in (glycans - set(tree.keys())):
        temp_new_glycans = []
        for k in glycan_motifs(glycan).keys():
            temp = glycan_motifs(glycan)
            temp[k] = temp[k]-1
            order = {'HexNAc':0, 'Hex': 1, 'Fuc': 2, 'NeuAc': 3, 'NeuGc': 4}
            temp_new_glycans.append(''.join(['{}({})'.format(i,j) for i,j in sorted(temp.items(), key=lambda x: order[x[0]]) if j!=0]))
        if any([temp_new_glycan in glycans for temp_new_glycan in temp_new_glycans]):
            temp_new_glycans = [temp_new_glycan for temp_new_glycan in temp_new_glycans if temp_new_glycan in glycans]
        new_glycans = new_glycans.union(set(temp_new_glycans)) 
    new_glycans = new_glycans - set([''])
    new_glycans = set([new_glycan for new_glycan in new_glycans if 'HexNAc' in new_glycan])
    glycans = glycans.union(new_glycans)
    mod_motifs = {}
    for glycan in glycans:
        mod_motifs[glycan]=[[k, str(v)] for k,v in glycan_motifs(glycan).items()]
    tree = tree_builder(mod_motifs)

In [ ]:
need_fuc_variant = []
for k in tree.keys():
    if 'Fuc' in k:
        foo=True
        keys = [k]
        while foo:
            try:
                need_fuc_variant+=[i for j in keys for i in tree[j]]
                keys = [i for j in keys for i in tree[j]]
            except:
                foo=False
new_glycans=set()
for i in set([i for i in need_fuc_variant if 'Fuc' not in i]):
    temp = glycan_motifs(i)
    temp.update({'Fuc':1})
    order = {'HexNAc':0, 'Hex': 1, 'Fuc': 2, 'NeuAc': 3, 'NeuGc': 4}
    new_glycans.add(''.join(['{}({})'.format(l,m) for l,m in sorted(temp.items(), key=lambda x: order[x[0]]) if m!=0]))
    #if 'Hex(' in i:
        #new_glycans.add(i[:15]+'Fuc(1)'+i[15:])
    #else:
        #new_glycans.add(i[:9]+'Fuc(1)'+i[9:])
        
mod_motifs = {}
for glycan in (glycans.union(new_glycans)):
    mod_motifs[glycan]=[[k, str(v)] for k,v in glycan_motifs(glycan).items()]
tree = tree_builder(mod_motifs)

In [ ]:
source=[]
target=[]
for k,v in tree.items():
    for i in range(len(v)):
        source.append(k)
        target.append(v[i])

cyto_df = pd.DataFrame(columns=['source','target'])
cyto_df['source']=source
cyto_df['target']=target

cyto_df.to_csv('', index=False) # glycan_fragmentation_tree for cytoscape

import pickle
with open('', 'wb') as fp:
    pickle.dump(tree, fp) # glycan_fragmentation_tree for library builder

4. Spectral Library Builder and Precursor Library Builder

In [ ]:
import pandas as pd
import numpy as np
from pyteomics import mass

# oxonium ions
ox_d = {}
ox_d[0] = ['[HexNAc-C2H6O3]', '[+126.0550]', 'OX', float('126.0550'), int('1'), float('126.0550')]
ox_d[1] = ['[HexNAc-CH6O3]', '[+138.0550]', 'OX', float('138.0550'), int('1'), float('138.0550')]
ox_d[2] = ['[HexNAc-C2H4O2]', '[+144.0656]', 'OX', float('144.0656'), int('1'), float('144.0656')]
ox_d[3] = ['[Hex-H2O]', '[+145.0495]', 'OX', float('145.0495'), int('1'), float('145.0495')]
ox_d[4] = ['[Hex]', '[+163.0601]', 'OX', float('163.0601'), int('1'), float('163.0601')]
ox_d[5] = ['[HexNAc-2H2O]', '[+168.0655]', 'OX', float('168.0655'), int('1'), float('168.0655')]
ox_d[6] = ['[HexNAc-H2O]', '[+186.0761]', 'OX', float('186.0761'), int('1'), float('186.0761')]
ox_d[7] = ['[HexNAc]', '[+204.0867]', 'OX', float('204.0867'), int('1'), float('204.0867')]
ox_d[8] = ['[HexHexNAc]', '[+366.1395]', 'OX', float('366.1395'), int('1'), float('366.1395')]
ox_d[9] = ['[Neu5Ac-H2O]', '[+274.092]', 'OX', float('274.092'), int('1'), float('274.092')]
ox_d[10] = ['[Neu5Ac]', '[+292.103]', 'OX', float('292.103'), int('1'), float('292.103')]

ox_df = pd.DataFrame(ox_d).T
ox_df.columns=['Annotation', 'FragmentSeq', 'FragmentType', 'TheoFragmentMass', 'z', 'FragmentMZ']

In [ ]:
import pickle
with open('', 'rb') as input_file:
    glycan_frag_network = pickle.load(input_file) # glycan_fragmentation_tree for library builder

In [ ]:
def find_children(k, tree):
    children = []
    keys = [k]
    foo=True
    while foo:
        new_children=[]
        for j in keys:
            try:
                new_children+=[i for i in tree[j]]
            except:
                continue
        if len(new_children)>0:
            keys=new_children
            children+=new_children
        else:
            foo=False
    return set(children)
def glycan_motifs(glycan):
    motifs = {}
    for a in glycan.split(')')[:-1]:
        motifs[a.split('(')[0]]=int(a.split('(')[1])
    return motifs

In [ ]:
data = pd.read_csv('', index_col=0) # nodelist_data from previous section
data = data.dropna(subset=['glycoform'])
data['glycoform'] = [eval(i) for i in data['glycoform']]

mol_network_data = pd.read_csv('', index_col='name',low_memory=False).loc[data.index] # METABOLOMICS-SNETS-V2 output node information

In [ ]:
mono_mass = {'HexNAc':203.080,'Hex':162.053,'Fuc':146.058,'NeuAc':291.095,'NeuGc':307.090}

precursor_mzs = []
frag_mzs = []
rela_frag_ints = []
precursor_rts = []
frag_annotations = []
frag_types = []
frag_zs = []
theo_frag_masses = []
precursor_zs = []
peptide_seqs = []
precursor_annotations = []
decoys = []

for index in data.index:
    for precursor_peptide, precursor_glycan in data.loc[index]['glycoform']:
        precursor_rt = round(mol_network_data.loc[index]['RTMean']/60, 4)
        precursor_motifs = glycan_motifs(precursor_glycan)
        precursor_pep_mass = mass.calculate_mass(sequence=precursor_peptide.split('.')[1])
        precursor_pep_glycan_mass = precursor_pep_mass + sum([mono_mass[k]*int(v) for k,v in precursor_motifs.items()])
        #precursor_pep_glycan_mass = data.loc[index]['parent mass'] 
        #precursor_pep_mass = data.loc[index]['parent mass'] - sum([mono_mass[k]*int(v) for k,v in precursor_motifs.items()])
        for z in range(2,4+1):
            precursor_charge = z
            precursor_mz = (precursor_pep_glycan_mass+z)/z
            for glycan_frag in find_children(k=precursor_glycan,tree=glycan_frag_network):
                frag_motifs = glycan_motifs(glycan_frag)
                for i in range(1,z):
                    frag_mz = (precursor_pep_mass+sum([mono_mass[k]*int(v) for k,v in frag_motifs.items()])+i)/i
                    frag_charge = i
                    theo_frag_mass = precursor_pep_mass+sum([mono_mass[k]*int(v) for k,v in frag_motifs.items()])+1
                    precursor_mzs.append(precursor_mz)
                    frag_mzs.append(frag_mz)
                    rela_frag_ints.append(0.5)
                    precursor_rts.append(precursor_rt)
                    frag_annotations.append(glycan_frag)
                    frag_types.append('Y')
                    frag_zs.append(frag_charge)
                    theo_frag_masses.append(theo_frag_mass)
                    precursor_zs.append(precursor_charge)
                    peptide_seqs.append(precursor_peptide)
                    precursor_annotations.append(precursor_glycan)
                    decoys.append(0)
            #oxonium ions
            for ox_ion in ox_df.index:
                if ox_df.loc[ox_ion]['Annotation'] == '[Hex-H2O]' or ox_df.loc[ox_ion]['Annotation'] == '[Hex]':
                    try:
                        if precursor_motifs['HexNAc']==2 and precursor_motifs['Hex']>=4:
                            frag_mz = ox_df.loc[ox_ion]['FragmentMZ']
                            glycan_frag = ox_df.loc[ox_ion]['Annotation']
                            frag_charge = ox_df.loc[ox_ion]['z']
                            theo_frag_mass = ox_df.loc[ox_ion]['TheoFragmentMass']
                            #print(precursor_glycan, glycan_frag, frag_mz, frag_charge)
                            precursor_mzs.append(precursor_mz)
                            frag_mzs.append(frag_mz)
                            rela_frag_ints.append(0.5)
                            precursor_rts.append(precursor_rt)
                            frag_annotations.append(glycan_frag)
                            frag_types.append('OX')
                            frag_zs.append(frag_charge)
                            theo_frag_masses.append(theo_frag_mass)
                            precursor_zs.append(precursor_charge)
                            peptide_seqs.append(precursor_peptide)
                            precursor_annotations.append(precursor_glycan)
                            decoys.append(0)
                    except:
                        continue
                if ox_df.loc[ox_ion]['Annotation'] == '[HexHexNAc]':
                    try:
                        if precursor_motifs['HexNAc']>2 and precursor_motifs['Hex']>3:
                            frag_mz = ox_df.loc[ox_ion]['FragmentMZ']
                            glycan_frag = ox_df.loc[ox_ion]['Annotation']
                            frag_charge = ox_df.loc[ox_ion]['z']
                            theo_frag_mass = ox_df.loc[ox_ion]['TheoFragmentMass']
                            #print(precursor_glycan, glycan_frag, frag_mz, frag_charge)
                            precursor_mzs.append(precursor_mz)
                            frag_mzs.append(frag_mz)
                            rela_frag_ints.append(0.5)
                            precursor_rts.append(precursor_rt)
                            frag_annotations.append(glycan_frag)
                            frag_types.append('OX')
                            frag_zs.append(frag_charge)
                            theo_frag_masses.append(theo_frag_mass)
                            precursor_zs.append(precursor_charge)
                            peptide_seqs.append(precursor_peptide)
                            precursor_annotations.append(precursor_glycan)
                            decoys.append(0)
                    except:
                        continue
                if ox_df.loc[ox_ion]['Annotation'] == '[Neu5Ac-H2O]' or ox_df.loc[ox_ion]['Annotation'] == '[Neu5Ac]':
                    if 'NeuAc' in precursor_glycan or 'NeuGc' in precursor_glycan:
                        frag_mz = ox_df.loc[ox_ion]['FragmentMZ']
                        glycan_frag = ox_df.loc[ox_ion]['Annotation']
                        frag_charge = ox_df.loc[ox_ion]['z']
                        theo_frag_mass = ox_df.loc[ox_ion]['TheoFragmentMass']
                        #print(precursor_glycan, glycan_frag, frag_mz, frag_charge)
                        precursor_mzs.append(precursor_mz)
                        frag_mzs.append(frag_mz)
                        rela_frag_ints.append(0.5)
                        precursor_rts.append(precursor_rt)
                        frag_annotations.append(glycan_frag)
                        frag_types.append('OX')
                        frag_zs.append(frag_charge)
                        theo_frag_masses.append(theo_frag_mass)
                        precursor_zs.append(precursor_charge)
                        peptide_seqs.append(precursor_peptide)
                        precursor_annotations.append(precursor_glycan)
                        decoys.append(0)
                elif ox_df.loc[ox_ion]['Annotation'] == '[HexNAc-C2H6O3]' or \
                ox_df.loc[ox_ion]['Annotation'] == '[HexNAc-CH6O3]' or \
                ox_df.loc[ox_ion]['Annotation'] == '[HexNAc-C2H4O2]' or \
                ox_df.loc[ox_ion]['Annotation'] == '[HexNAc-2H2O]' or \
                ox_df.loc[ox_ion]['Annotation'] == '[HexNAc-H2O]' or \
                ox_df.loc[ox_ion]['Annotation'] == '[HexNAc]':
                    frag_mz = ox_df.loc[ox_ion]['FragmentMZ']
                    glycan_frag = ox_df.loc[ox_ion]['Annotation']
                    frag_charge = ox_df.loc[ox_ion]['z']
                    theo_frag_mass = ox_df.loc[ox_ion]['TheoFragmentMass']
                    #print(precursor_glycan, glycan_frag, frag_mz, frag_charge)
                    precursor_mzs.append(precursor_mz)
                    frag_mzs.append(frag_mz)
                    rela_frag_ints.append(0.5)
                    precursor_rts.append(precursor_rt)
                    frag_annotations.append(glycan_frag)
                    frag_types.append('OX')
                    frag_zs.append(frag_charge)
                    theo_frag_masses.append(theo_frag_mass)
                    precursor_zs.append(precursor_charge)
                    peptide_seqs.append(precursor_peptide)
                    precursor_annotations.append(precursor_glycan)
                    decoys.append(0)

In [ ]:
spectral_lib = pd.DataFrame()

spectral_lib['PrecursorMz'] = precursor_mzs
spectral_lib['FragmentMz'] = frag_mzs
spectral_lib['RelativeFragmentIntensity'] = rela_frag_ints
spectral_lib['RetentionTime'] = precursor_rts
spectral_lib['Annotation'] = frag_annotations
spectral_lib['FragmentType'] = frag_types
spectral_lib['FragmentCharge'] = frag_zs
spectral_lib['TheoreticalFragmentMass'] = theo_frag_masses
spectral_lib['PrecursorCharge'] = precursor_zs
spectral_lib['PeptideSequence'] = peptide_seqs
spectral_lib['PrecursorAnnotation'] = precursor_annotations
spectral_lib['Decoy'] = decoys

spectral_lib.to_csv('', index=False,sep='\t')

In [ ]:
precursor_lib = spectral_lib[['PeptideSequence','PrecursorAnnotation','PrecursorMz','PrecursorCharge','PrecursorMz','PrecursorCharge']]
precursor_lib.columns = ['Molecule List Name','Molecule Name','Precursor Mz','Precursor Charge','Product Mz','Product Charge']

precursor_lib = precursor_lib.drop_duplicates()
precursor_lib['Molecule List Name']=[i.split('.')[1] for i in precursor_lib['Molecule List Name']]
precursor_lib.insert(6,'Fragment Ion', 'precursor')

precursor_lib.to_csv('', index=False)